In [19]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (15,9)
import json
import sqlite3
import pprint
import requests
import facebook
import datetime

In [20]:
CACHE_FNAME = "206_finalproject.json"
try:
    cache_file = open(CACHE_FNAME,'r')
    cache_contents = cache_file.read()
    cache_file.close()
    CACHE_DICTION = json.loads(cache_contents)
except:
    CACHE_DICTION = {}
    
def performsearch(search_term, api_query_function):
    cachekey = api_query_function.__name__ + "_" + search_term
    if cachekey in CACHE_DICTION:
        toreturn = CACHE_DICTION[cachekey]
    else:
        toreturn = api_query_function(search_term)
        CACHE_DICTION[cachekey] = toreturn
        writefile = open(CACHE_FNAME,"w")
        writefile.write(json.dumps(CACHE_DICTION))
        writefile.close()
    return toreturn

In [21]:
# import urllib
# import urllib.parse
# import subprocess
# import warnings

# FACEBOOK_APP_ID = "554002574961610"
# FACEBOOK_APP_SECRET = "3b06c813cb46e53a940e6985a64bcd45"


# #facebook_token = facebook_ID + "|" + facebook_secret

# oauth_args = dict(client_id     = FACEBOOK_APP_ID,
#                   client_secret = FACEBOOK_APP_SECRET,
#                   grant_type    = 'client_credentials')
# oauth_curl_cmd = ['curl',
#                   'https://graph.facebook.com/oauth/access_token?' + urllib.parse.urlencode(oauth_args)]
# oauth_response = subprocess.Popen(oauth_curl_cmd,
#                                   stdout = subprocess.PIPE,
#                                   stderr = subprocess.PIPE).communicate()[0]

# try:
#     oauth_access_token = urllib.parse.parse_qs(str(oauth_response))['access_token'][0]
# except KeyError:
#     print('Unable to grab an access token!')
#     exit()

# graph = facebook.GraphAPI(oauth_access_token)


# #graph = facebook.GraphAPI(facebook_token)

# profile = graph.get_object("me")
# friends = graph.get_connections("me", "friends")

# #url_parameters = {'format': 'json'}
# #baseurl = 'http://services.faa.gov/airport/status/'
# #airport = 'DTW'
# #airport_response = requests.get(baseurl+airport, params=url_parameters)

In [22]:
instagram_access_token = "226042116.1677ed0.d72827a090bd4641a961139b7f04530b"

def query_instagram_directly(search_term):
    url = 'https://api.instagram.com/v1/users/self/media/recent/?access_token={}'.format(instagram_access_token)
    response_string = requests.get(url).text
    posts = json.loads(response_string)['data']
    #user_id = posts[0]['user']['id']
    return posts

def query_instagram():
    return performsearch("blah", query_instagram_directly)

x = query_instagram()
pprint.pprint(x)

[{'attribution': None,
  'caption': {'created_time': '1511311217',
              'from': {'full_name': 'alyrosenberg',
                       'id': '226042116',
                       'profile_picture': 'https://scontent.cdninstagram.com/t51.2885-19/s150x150/22278167_121148855242753_2329136837357993984_n.jpg',
                       'username': 'alyrosenberg'},
              'id': '17885210626136291',
              'text': 'Missed u'},
  'comments': {'count': 6},
  'created_time': '1511311217',
  'filter': 'Normal',
  'id': '1653320781265307320_226042116',
  'images': {'low_resolution': {'height': 339,
                                'url': 'https://scontent.cdninstagram.com/t51.2885-15/e35/p320x320/23735329_935904079904602_8738584715784617984_n.jpg',
                                'width': 320},
             'standard_resolution': {'height': 678,
                                     'url': 'https://scontent.cdninstagram.com/t51.2885-15/sh0.08/e35/p640x640/23735329_935904079904602_873

                               'profile_picture': 'https://scontent.cdninstagram.com/t51.2885-19/s150x150/21688820_160785807832706_3633246177241071616_n.jpg',
                               'username': 'sophie_brown1'}},
                     {'position': {'x': 0.328, 'y': 0.7256312849162011},
                      'user': {'full_name': 'Ali Kraft',
                               'id': '35655761',
                               'profile_picture': 'https://scontent.cdninstagram.com/t51.2885-19/s150x150/20688031_475017139523508_654973506093580288_a.jpg',
                               'username': 'ali_kraft'}},
                     {'position': {'x': 0.1973333333333333,
                                   'y': 0.8141229050279329},
                      'user': {'full_name': 'Alexa Fox',
                               'id': '177301221',
                               'profile_picture': 'https://scontent.cdninstagram.com/t51.2885-19/s150x150/21372829_129418521016122_2059326145714716672_a.jpg

In [23]:
def query_github_directly(github_user_id = "alyrosenberg"):
    url = 'https://api.github.com/users/{}/events'.format(github_user_id)
    response_string = requests.get(url).text
    github_events = json.loads(response_string)
    return github_events

def query_github(github_user_id = "alyrosenberg"):
    return performsearch(github_user_id, query_github_directly)


In [24]:
#create connection to local SQLite database
conn = sqlite3.connect('206_finalproject.sqlite')
cur = conn.cursor()

#Dropping Tweets table if it exists then initilizing Tweets table
cur.execute('DROP TABLE IF EXISTS GitHub_Events')
cur.execute('CREATE TABLE GitHub_Events(id INTEGER PRIMARY KEY, created_at TIMESTAMP, type TEXT, repo_name TEXT)')

GitHub_Events = query_github()
for event in GitHub_Events:
    cur.execute('INSERT INTO GitHub_Events(id, created_at, type, repo_name) VALUES (?, ?, ?, ?)', 
                (event['id'], event['created_at'], event['type'], event["repo"]['name']))

conn.commit()

In [38]:
#Dropping Tweets table if it exists then initilizing Tweets table
cur.execute('DROP TABLE IF EXISTS Instagram_Posts')
cur.execute('CREATE TABLE Instagram_Posts(id TEXT PRIMARY KEY, created_at TIMESTAMP, caption_text TEXT, likes INTEGER, lat INTEGER, lng INTEGER)')

Instagram_Posts = query_instagram()
for post in Instagram_Posts:
    print (post['id'])
    print (post['created_time'])
    print (post['caption']['text'])
    print (post["likes"]['count'])
    #print (post["location"]['latitude'])
    try:
        lattemp = post["location"]['latitude']
        lngtemp = post["location"]['longitude']
    except:
        lattemp = ""
        lngtemp = ""
    if lattemp == None:
        lattemp = ""
    if lngtemp == None:
        lngtemp = ""
    formatteddatetime = datetime.datetime.fromtimestamp( int(post['created_time']) ).strftime('%Y-%m-%d %H:%M:%S')
    print (post['id'], formatteddatetime, post['caption']['text'], post["likes"]['count'], post["location"]['latitude'], post["location"]['longitude'])
    cur.execute('INSERT INTO Instagram_Posts(id, created_at, caption_text, likes, lat, lng) VALUES (?, ?, ?, ?, ?, ?)', (post['id'], formatteddatetime, post['caption']['text'], post["likes"]['count'], lattemp, lngtemp))
    
conn.commit()

1653320781265307320_226042116
1511311217
Missed u
532
1653320781265307320_226042116 2017-11-21 19:40:17 Missed u 532 40.7590802 -73.9728582
1651604675911595773_226042116
1511106641
Next stop: Barcelona
562
1651604675911595773_226042116 2017-11-19 10:50:41 Next stop: Barcelona 562 43.073182648942 -89.3988378321
1642235257343529660_226042116
1509989720
Visiting day
609


TypeError: 'NoneType' object is not subscriptable